In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy.stats import norm
from math import *

In [2]:
class BlackScholes:
    
        
    def ft_bs_pricing(self, Option_type,S,K,T,r,rf,sigma):
        
        global d1
        global d2
        d1 = (log(S / K) + (r - rf + sigma ** 2 / 2) * T) / (sigma * sqrt(T)) 
        d2 = d1 - sigma* sqrt(T) 
        
        if Option_type == "Call":
            option_price = S* exp(-rf*T)* norm.cdf(d1) - K * exp(-r*T)*norm.cdf(d2)
            return option_price
        elif Option_type == "Put":
            option_price = K* exp(-r*T)* norm.cdf(-d2) - S * exp(-rf*T)*norm.cdf(-d1)
            return option_price
        else:
            return "incorrect parameters"
       
   
    def ft_delta(self, Option_type,S,K,T,r,rf,sigma):
        if Option_type == "Call":
            delta = exp(-rf*T) * norm.cdf(d1)   
        elif Option_type == "Put":
            delta = exp(-rf*T) * (norm.cdf(d1)-1)
        return delta
    
    
    def ft_gamma(self, Option_type,S,K,T,r,rf,sigma):
        return (norm.pdf(d1)*exp(-rf*T) )/ ( S*sigma*sqrt(T) )
    
    def ft_vega(self, Option_type,S,K,T,r,rf,sigma):
        return S*sqrt(T)*norm.pdf(d1)*exp(-rf*T)
    
    def ft_theta(self, Option_type,S,K,T,r,rf,sigma):
        a = (-S *norm.pdf(d1) *sigma* exp(-rf*T)) /(2*sqrt(T))
        b = rf * S * norm.cdf(d1) * exp(-rf*T) 
        c = r * K * exp(-r*T)* norm.cdf(d2)
        b2 = rf * S * norm.cdf(-d1) * exp(-rf*T) 
        c2 = r * K * exp(-r*T)* norm.cdf(-d2)
        if Option_type == "Call":
             theta = a+b-c   
        elif Option_type == "Put":
            theta = a-b2+c2 
        return theta
    
    def ft_rho_dom(self, Option_type,S,K,T,r,rf,sigma):
        if Option_type == "Call":
            rho_dom = K * T *exp(-r*T) * norm.pdf(d2)   
        elif Option_type == "Put":
            rho_dom = -K * T *exp(-r*T) * norm.pdf(-d2)  
        return rho_dom
    
    def ft_rho_fgn(self, Option_type,S,K,T,r,rf,sigma):
        if Option_type == "Call":
            rho_fgn = -S * T *exp(-rf*T) * norm.pdf(d1)   
        elif Option_type == "Put":
            rho_fgn = S * T *exp(-rf*T) * norm.pdf(-d1)  
        return rho_fgn
    
    def ft_vanna(self, Option_type,S,K,T,r,rf,sigma):
        return -exp(-rf*T) * norm.cdf(d1) * d2 / sigma 
    
    
    def __init__(self,Option_type,S,K,T,r,rf,sigma):
        
        self.option_type = Option_type
        self.asset_price = S
        self.strike = K
        self.time_to_exp = T
        self.d_rate = r
        self.f_rate = rf
        self.asset_volatility = sigma
        
        self.price = self.ft_bs_pricing(Option_type,S,K,T,r,rf,sigma)
        
        
        self.delta = self.ft_delta(Option_type,S,K,T,r,rf,sigma)
        self.gamma = self.ft_gamma(Option_type,S,K,T,r,rf,sigma)
        self.theta = self.ft_theta(Option_type,S,K,T,r,rf,sigma)
        self.vega = self.ft_vega(Option_type,S,K,T,r,rf,sigma)
        self.rho_dom = self.ft_rho_dom(Option_type,S,K,T,r,rf,sigma)
        self.rho_fgn = self.ft_rho_fgn(Option_type,S,K,T,r,rf,sigma)
        self.vanna = self.ft_vanna(Option_type,S,K,T,r,rf,sigma)
       

In [43]:
# test de la fonction
Option_type = "Call"
N = -929000
S = 1.2034
K = 1.1826
r = 0.527843/100
rf = 0.137174/100
sigma = 8.21/100

T1 = '19/04/2021'
T2 = '22/04/2021'
T = pd.to_datetime(T2) - pd.to_datetime(T1)
T  = T.days /360

bs = BlackScholes(Option_type,S,K,T,r,rf,sigma)
print( 'price = ', round(N*bs.price,2) )
print( 'delta = ', round(bs.delta,2) )
print( 'gamma = ', round(bs.gamma,2) )
print( 'vega = ', round(bs.vega,2) )
print( 'theta = ', round(bs.theta,2) )

price =  -19386.53
delta =  0.99
gamma =  2.9
vega =  0.0
theta =  -0.02


## Risked based method

### PnL spot
On sait que :
$ V(S_1) = V(S_0) + \frac{dV}{dS} * (S_1-S_0) + \frac{1}{2!}\frac{d^2V}{dS^2} * (S_1-S_0)^2 + \frac{1}{3!} \frac{d^3V}{dS^3} * (S_1-S_0)^3 + $... 
Avec $\frac{dV}{dS} = \Delta$ de l'option, $\frac{d^2V}{dS^2} = \gamma$ de l'option.

Comme on ne connait pas $\frac{d^3V}{dS^3}$, on  peut l'estimer par les différentielles : 

$\frac{d^3V}{dS^3} = \frac{d}{dS}\left(\frac{d^2V}{dS^2}\right) = \frac{d}{dS}\left(\gamma\right) =~ \frac{\Delta \gamma}{\Delta S}$

Ainsi on peut ecrire:

$ V(S_1) = V(S_0) + \Delta * (S_1-S_0) + \frac{1}{2!}\gamma * (S_1-S_0)^2 + \frac{1}{3!}\frac{\gamma_1 - \gamma_0}{S_1-S_0} * (S_1-S_0)^3 + $... 

D'où :

$ V(S_1) = V(S_0) + \Delta * (S_1-S_0) + \frac{1}{2}\gamma * (S_1-S_0)^2 + \frac{1}{6}(\gamma_1 - \gamma_0) * (S_1-S_0)^2 + $... 


In [13]:
bs = BlackScholes("Call",100,100,1,0.04,0.08,0.30)

price1 = bs.price
delta1 = bs.delta 
gamma1 = bs.gamma 
vega1 = bs.vega
theta1 = bs.theta
rho_dom1 = bs.rho_dom
rho_fgn1 = bs.rho_fgn

bs2 = BlackScholes("Call",140,100,0.9,0.06,0.06,0.32) 
price2 = bs2.price

var_spot = 40
var_vol = 0.02
var_time = -0.1
var_rho_dom = 0.02
var_rho_fgn = -0.02

# taylor series approx : f(x) = f(x0) + f'(x0)(x-x0) + f''(x0)(x-x0)**2 +... 
PnL_spot =  delta1*var_spot + 0.5*gamma1*var_spot**2 + (1/6)*(bs2.gamma -bs.gamma)*var_spot**2 
PnL_vol = vega1*var_vol
PnL_time = -theta1*var_time
PnL_dom = rho_dom1*var_rho_dom 
PnL_fgn = rho_fgn1*var_rho_fgn
PnL = PnL_spot  + PnL_time + PnL_vol + PnL_dom + PnL_fgn

Real_PnL = price2 - price1
print("Total PnL",PnL)
print("Observed PnL",Real_PnL)
print(end = '\n')

print("Impact of price change", round(PnL_spot/Real_PnL*100,2),"%")
print("Impact of time", round(PnL_time/Real_PnL*100,2),"%")
print("Impact of volatility", round(PnL_vol/Real_PnL*100,2),"%")
print("Impact of domestic rate", round(PnL_dom/Real_PnL*100,2),"%")
print("Impact of foreign rate", round(PnL_fgn/Real_PnL*100,2),"%")
print("Unexplained PnL",round( (Real_PnL - PnL) /Real_PnL*100,2),"%") 


Total PnL 28.20782490561346
Observed PnL 30.73023269480405

Impact of price change 85.67 %
Impact of time -1.07 %
Impact of volatility 2.4 %
Impact of domestic rate 2.4 %
Impact of foreign rate 2.4 %
Unexplained PnL 8.21 %


## Step re-evaluation

In [12]:
Option_type = "Call"
K = 100

S1 = 100
T1 = 1
r1 = 0.04
rf1 = 0.08
sigma1 = 0.30

S2 = 140
T2 = 0.9
r2 = 0.06
rf2 = 0.06
sigma2 = 0.32

def attribution(Option_type,K, S1,T1,r1,rf1,sigma1, S2,T2,r2,rf2,sigma2):
    price_init = BlackScholes(Option_type,S1,K,T1,r1,rf1,sigma1).price 
    price_fin = BlackScholes(Option_type,S2,K,T2,r2,rf2,sigma2).price  
    price_spot = BlackScholes(Option_type,S2,K,T1,r1,rf1,sigma1).price 
    price_vol = BlackScholes(Option_type,S1,K,T1,r1,rf1,sigma2).price 
    price_time = BlackScholes(Option_type,S1,K,T2,r1,rf1,sigma1).price 
    price_dom = BlackScholes(Option_type,S1,K,T1,r2,rf1,sigma1).price 
    price_fgn = BlackScholes(Option_type,S1,K,T1,r1,rf2,sigma1).price 
    
    Real_PnL = price_fin-price_init
    PnL_spot = price_spot-price_init
    PnL_time = price_time-price_init
    PnL_vol = price_vol-price_init
    PnL_dom = price_dom-price_init
    PnL_fgn = price_fgn-price_init
    PnL = PnL_spot + PnL_time + PnL_vol + PnL_dom + PnL_fgn
    Unexplained_PnL = Real_PnL - PnL
    
    print("Total PnL",PnL)
    print("Observed PnL",Real_PnL)
    print(end = '\n')

    print("Impact of price change", round(PnL_spot/Real_PnL*100,2),"%")
    print("Impact of time", round(PnL_time/Real_PnL*100,2),"%")
    print("Impact of volatility", round(PnL_vol/Real_PnL*100,2),"%")
    print("Impact of domestic rate", round(PnL_dom/Real_PnL*100,2),"%")
    print("Impact of foreign rate", round(PnL_fgn/Real_PnL*100,2),"%")
    print("Unexplained PnL",round( Unexplained_PnL /Real_PnL*100,2),"%") 


In [13]:
attribution(Option_type,K, S1,T1,r1,rf1,sigma1, S2,T2,r2,rf2,sigma2)

Total PnL 28.663657246276976
Observed PnL 30.73023269480405

Impact of price change 86.36 %
Impact of time -1.12 %
Impact of volatility 2.4 %
Impact of domestic rate 2.48 %
Impact of foreign rate 3.16 %
Unexplained PnL 6.72 %
